## Steady state Flux Analysis

In [14]:
import pandas as pd
import glob

#Get Metabolite info file into a pandas dataframe
m_df =pd.read_csv("./MetaboliteInfo_v5.5.csv", index_col="Metabolite")

# Get a list of Enrichment and Sampleinfo files starting with "E" and "S" respectively
e_files = glob.glob("E*.csv")
s_files = glob.glob("S*.csv")

### Sample info file

In [15]:
# Read each file into a pandas dataframe
s_dfs = []
for filename in s_files:
    df = pd.read_csv(filename,index_col=['Original.Sample.Name'])
    s_dfs.append(df)

# Concatenate all dataframes into one dataframe
s_df = pd.concat(s_dfs)

# Get columns with names 'Biol.Rep' and'Tracer'
cols = ['Biol.Rep','Tracer']

# Get column names starting with "GroupLevel"
g_cols = [col for col in s_df.columns if col.startswith("GroupLevel")]

# Filter columns from g_cols that are not completely empty
nonempty_g_cols = [col for col in g_cols if not s_df[col].isnull().all()]

# Combine cols and nonempty_g_cols to get the final list of columns
final_cols = cols + nonempty_g_cols

# Extract desired columns from the sample info dataframe
new_s_df = s_df[final_cols].rename(index=str.lower)

### Combining Sampleinfo and Enrichment file

In [16]:
##Read each file into a pandas dataframe and map the sample name to grouplevel
from functools import reduce

e_dfs = []
for filename in e_files:
    df = pd.read_csv(filename,index_col='compoundid')
    merged_df = pd.concat([df, new_s_df], axis=1, join="inner").set_index(final_cols)
    e_dfs.append(merged_df)

combined_df = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True,how="inner"), e_dfs)

### User defined functions 

In [17]:
def cid_query(mdf,cdf,metname,tracer,*arg):
    '''
    Returns the list of compoundids from metabolite info file from matching the desired fragment and tracer of interest

    Parameters:
    mdf (pandas.DataFrame): metabolite info dataframe
    cdf (pandas.DataFrame): combined sample info and enrichment dataframe
    metname (str): name of the metabolite of interest
    tracer (str): name of the tracer of interest
    arg: fragment information e.g: "M+1" or 1,1

    Returns:
    list: A list containing the compound ids of the matching columns.

    If no matching columns are found, returns empty list
    '''
    if len(arg) == 2:
        return mdf[(mdf.index==metname)&(mdf['Parent.Label']==arg[0])&(mdf['Daughter.Label']==arg[1])].compoundId.tolist()
    elif len(arg) == 1:
        return mdf[(mdf.index==metname)&(mdf['Mass']==arg[0])].compoundId.tolist()

def sum_generator(mdf,cdf,metname,tracer,*arg):
    '''
    Returns a dataframe with Enrichment from matching the desired fragment and tracer of interest

    Parameters:
    mdf (pandas.DataFrame): metabolite info dataframe
    cdf (pandas.DataFrame): combined sample info and enrichment dataframe
    metname (str): name of the metabolite of interest
    tracer (str): name of the tracer of interest
    arg: fragment information e.g: "M+1" or 1,1

    Returns:
    DataFrame: A dataframe containing the enrichment values

    If no matching columns are found, returns empty dataframe
    '''
    df = cdf[cdf.index.get_level_values(1).str.replace('-', '') == tracer]
    cid_list = cid_query(mdf,cdf,metname,tracer,*arg)
    matching_cids = [col for col in df.columns if col in cid_list]
    if len(matching_cids) == 0:
        print('No enrichment found for the given input '+str(metname,tracer,arg))
        return None
    else:        
        return df[matching_cids].sum(axis=1).droplevel(level=1)

    
def sum_query(code,mdf,cdf):
    """
    Returns a function call by parsing the code letter and calls the sum_generator function.

    Parameters:
    code (str): code letter containing the tracer, metabolite and fragemnt information
    mdf (pandas.DataFrame): metabolite info dataframe
    cdf (pandas.DataFrame): combined sample info and enrichment dataframe

    Returns:
    Calls a function: Calls sum_generator
    """
    arg_frag = str(code[5:])
    arg_t = str(t_ind[code[0:2]])
    arg_m = str(m_ind[code[2:5]])
    if arg_frag.startswith("M"):
        new_arg_frag = arg_frag[0]+"+"+arg_frag[1]
        return sum_generator(mdf,cdf,arg_m,arg_t,new_arg_frag)
    else:
        arg_P = int(arg_frag[1])
        arg_D = int(arg_frag[3])
        return sum_generator(mdf,cdf,arg_m,arg_t,arg_P,arg_D)

def df_generator(listofmet,mdf,cdf):
    """
    Returns a dataframe by combining all the enrichment data from the list of inputs

    Parameters:
    listofmet (list): List of code letters containing the tracer, metabolite and fragemnt information of interest
    mdf (pandas.DataFrame): metabolite info dataframe
    cdf (pandas.DataFrame): combined sample info and enrichment dataframe

    Returns:
    DataFrame: A dataframe containing the enrichment values
    """
    #print([sum_query(i,mdf,cdf).index for i in listofmet])
    out = pd.concat([sum_query(i,mdf,cdf) for i in listofmet],keys=listofmet,axis=1)
    return out

### Steady state flux ratio calculation

In [18]:
# Assigning a 2 letter index for each Tracers
t_ind = {"qq":"13CGln","4g":"13CGlc","5g":"513CGlc"}

# Assigning a 3 letter index for each Metabolites (Add new metabolites if needed)
m_ind = {"Asp":"Aspartate_88","Gln":"Glutamine","Glu":"Glutamate_102","Mal":"Malate","Ala":"Alanine",
           "Cit":"Citrate_111","Pyr":"Pyruvate","PEP":"PEP","3PG":"3PG","Lac":"Lactate","Suc":"Succinate_73"}

#####Generating datframe with enrichments needed for calculation######
List = ["qqAspM1","qqAspM2","qqCitM1","qqCitM2","qqCitM3","qqGluM1","qqGluM2","qqGluM3","qqGlnM2","qqGlnM3","qqPyrM1",
        "4gPyrM1","qqMalM1","qqMalM2","4gAspM1","4gAspP1D0","4gAspP1D1","4gGluM1","4gGlnM1","4gCitM1","4gMalM1","4gPEPM1","4g3PGM1",
        "4gMalP1D1","5gGluP1D0","5gGluP1D1","5gGluP2D1","5gGluP2D2","5gGluP3D2","5gGluP3D3","5gPyrM1","5gPEPM1",
        "5g3PGM1","5gLacM1","5gGluM1","5gGluM2","5gGluM3","5gGlnM1","5gGlnM2","5gGlnM3","4gCitP1D0","4gCitP1D1","qqCitP1D0","qqCitP1D1","qqCitP2D1","qqCitP2D2",
       "4gLacM1","qqLacM1","5gMalM1","5gMalM2","5gAspP1D0","5gAspP1D1","5gAspM1","5gAspM2","5gCitM1","5gCitM2","5gCitP2D1",
        "5gCitP2D2","5gCitP1D1","5gCitP1D0","5gCitP2D1","5gCitP2D2","5gCitP3D2","5gCitP3D3","qqAspP2D1","5gSucM1",
       "5gSucP1D0","5gSucP1D1","qq3PGM1","qqAspP1D0","qqAspP1D1","5gAspP2D1","5gAspP2D2"]

DF = df_generator(List,m_df,combined_df)




In [19]:
#CITRATE NODE
DF["a1"] = DF["qqGluM2"]/DF["qqCitM3"]
DF["a3"] = (DF["qqMalM2"]-DF["qqGluM2"]) / (DF["qqGluM1"]-DF["qqGluM2"]-DF["qqCitM2"] * DF["a1"])
DF["a4"] = (DF["qqMalM2"]) / (DF["qqGluM1"]-DF["qqGluM2"]-DF["qqCitM2"] * DF["a1"])  
DF["qqGluM0"] = 1-DF["qqGluM1"]-DF["qqGluM2"]-DF["qqGluM3"]
DF["fIDHr"]=(DF["a4"]*DF["qqGluM0"]-DF["qqCitM1"]*DF["a1"]*DF["a4"]-DF["qqMalM1"]-DF["qqGluM1"]*DF["a4"])/(DF["a3"]*DF["qqGluM0"]+DF["qqGluM1"]-DF["qqMalM1"]-DF["qqGluM1"]*DF["a3"]-DF["qqCitM1"]*DF["a1"]*DF["a3"])
DF["fIDHr_average"] = DF["fIDHr"].groupby(level=[1]).transform("mean")
DF["qqBicarb"] = DF["a3"] - DF["a4"] * (1/ DF["fIDHr"])
DF["FCitDil"] = DF["a1"] * DF["qqBicarb"] * DF["fIDHr"]
DF["FCitDil_average"] = DF["FCitDil"].groupby(level=[1]).transform("mean")
DF["fCS"]=1-DF["fIDHr"]
DF["fCS_average"] = DF["fCS"].groupby(level=[1]).transform("mean")

DF.loc[:,["fIDHr","qqBicarb","FCitDil"]]


,,fIDHr,qqBicarb,FCitDil
Biol.Rep,GroupLevel1,,,
1.0,G2.5Q2,0.268815,0.073234,1.307418
2.0,G2.5Q2,0.250447,0.073904,1.222850
3.0,G2.5Q2,0.242699,0.083056,1.274133
4.0,G2.5Q2,0.224721,0.077301,1.194089
1.0,G5Q2,0.218380,0.074298,1.218196
2.0,G5Q2,0.215654,0.082277,1.199014
3.0,G5Q2,0.194968,0.088290,1.224948
4.0,G5Q2,0.170412,0.086698,1.231996
1.0,G7Q2,0.188839,0.084290,1.181137


In [20]:
#GLUTAMATE NODE            
DF["fIDH"] = DF["qqGluM1"] / ((DF["qqCitP1D0"]*2+(DF["qqCitP2D2"]-DF["qqCitP2D1"]))*DF["FCitDil"])
DF["fAAO"] = DF["qqGluM2"]/DF["qqGlnM2"] - (((DF["qqCitM3"]+DF["qqCitP2D1"]*2)*DF["FCitDil"])/DF["qqGlnM2"])*DF["fIDH"]   
DF["fAAO_#2"] = DF["qqGluM3"]/DF["qqGlnM3"]   
DF["fAADil"] = 1-DF["fIDH"]-DF["fAAO"]
DF["fIDH_average"]=DF["fIDH"].groupby(level=[1]).transform("mean")
DF["fAAO_average"]=DF["fAAO"].groupby(level=[1]).transform("mean")
DF["fAADil_average"]=DF["fAADil"].groupby(level=[1]).transform("mean")

DF.loc[:,["fIDH","fAAO","fAAO_#2","fAADil"]]


,,fIDH,fAAO,fAAO_#2,fAADil
Biol.Rep,GroupLevel1,,,,
1.0,G2.5Q2,0.273965,0.595368,0.542771,0.130668
2.0,G2.5Q2,0.300689,0.593082,0.587258,0.106228
3.0,G2.5Q2,0.275611,0.578327,0.581262,0.146062
4.0,G2.5Q2,0.273794,0.592970,0.539989,0.133236
1.0,G5Q2,0.338002,0.518327,0.542784,0.143670
2.0,G5Q2,0.364308,0.490725,0.510852,0.144966
3.0,G5Q2,0.343706,0.501633,0.495624,0.154662
4.0,G5Q2,0.329006,0.507318,0.481201,0.163676
1.0,G7Q2,0.384113,0.486916,0.493215,0.128971


In [21]:
# AcetylCoA NODE
DF["5gcoam1"]= DF["5gGluP2D1"] / (DF["5gGluP2D1"] + DF["5gGluP1D1"])
DF["fPDH"]=DF["5gcoam1"]/DF["5g3PGM1"]
DF["fPDH_average"]=DF["fPDH"].groupby(level=[1]).transform("mean")
DF.loc[DF["fPDH_average"] > 1, "fPDH_average"] = 1

DF.loc[:,["fPDH","fPDH_average"]]


,,fPDH,fPDH_average
Biol.Rep,GroupLevel1,,
1.0,G2.5Q2,0.788840,0.940321
2.0,G2.5Q2,1.051119,0.940321
3.0,G2.5Q2,0.816997,0.940321
4.0,G2.5Q2,1.104326,0.940321
1.0,G5Q2,1.178327,1.000000
2.0,G5Q2,1.116241,1.000000
3.0,G5Q2,1.188112,1.000000
4.0,G5Q2,1.229643,1.000000
1.0,G7Q2,1.122417,1.000000


In [22]:
#Citrate Precursor Node
DF["4gGluM0"]=1-DF["4gGluM1"]

DF["4gBicarb"] = (DF["4gCitM1"] * DF["FCitDil_average"] - DF["4gGluM1"]*DF["fIDHr_average"] - DF["4gMalM1"] * (1-DF["fIDHr_average"]))/(DF["4gGluM0"]*DF["fIDHr_average"]-DF["4gGluM1"]*DF["fIDHr_average"])
DF.loc[DF["4gBicarb"] < 0, "4gBicarb"] = 0

DF["a5"] = DF["4gMalM1"] / (DF["4g3PGM1"]*(1-DF["4gBicarb"]) + DF["4gBicarb"]*(1-DF["4g3PGM1"]))
DF["a5_average"] = DF["a5"].groupby(level=[1]).transform("mean")
DF["a6"] = DF["4gCitM1"]* DF["FCitDil_average"] / (DF["4g3PGM1"]*(1-DF["4gBicarb"]) + DF["4gBicarb"]*(1-DF["4g3PGM1"]))
DF["a6_average"] = DF["a6"].groupby(level=[1]).transform("mean")


DF["fACLy"] = (DF["qqMalM2"] - DF["qqGluM3"] + DF["qqGluM3"] * DF["a5_average"])/(DF["qqGluM3"] * DF["a6_average"] + DF["qqCitM2"] - DF["qqGluM3"]) 
DF.loc[DF["fACLy"] < 0, "fACLy"] = 0
DF["fACLy_average"] = DF["fACLy"].groupby(level=[1]).transform("mean")
DF["fPC&MEr"] = DF["a5"] - DF["a6"] * DF["fACLy_average"]
DF["fTCA"] = 1- DF["fPC&MEr"] - DF["fACLy"]
DF["fPC&MEr_average"] = DF["fPC&MEr"].groupby(level=[1]).transform("mean")
DF["fTCA_average"] = DF["fTCA"].groupby(level=[1]).transform("mean")
DF["fMDHr"]=DF["4gAspP1D0"]/DF["4gAspM1"]
DF["fMDHr_average"]=DF["fMDHr"].groupby(level=[1]).transform("mean")

DF.loc[:,["fACLy","fPC&MEr","fTCA","fMDHr","4gBicarb"]]


,,fACLy,fPC&MEr,fTCA,fMDHr,4gBicarb
Biol.Rep,GroupLevel1,,,,,
1.0,G2.5Q2,0.156269,0.012595,0.831136,0.268812,0.001931
2.0,G2.5Q2,0.027320,0.012872,0.959808,0.257327,0.000000
3.0,G2.5Q2,0.158707,0.033040,0.808253,0.305530,0.000000
4.0,G2.5Q2,0.132806,0.027430,0.839764,0.534837,0.000000
1.0,G5Q2,0.081758,0.049710,0.868532,0.343757,0.006693
2.0,G5Q2,0.145773,0.058792,0.795434,0.370601,0.000000
3.0,G5Q2,0.249123,0.051316,0.699561,0.359989,0.006179
4.0,G5Q2,0.287800,0.041173,0.671028,0.281335,0.035809
1.0,G7Q2,0.148405,0.146557,0.705037,0.386470,0.033072


In [23]:
#Gln node
#na13C,na2H,na15N,na17O,na18O = 0.0107,0.000115,0.00368,0.00038,0.00205

NAGlnM3 = .98*.98*(0.0107*3+0.000115*10+0.00368*2+0.00038*3)


DF["AAS/AAO"]=(DF["qqGlnM2"]-(1-NAGlnM3))/(DF["qqGluM2"]-(1-NAGlnM3))
DF["AAS/AAO_average"]=DF["AAS/AAO"].groupby(level=[1]).transform("mean")
DF.loc[:,["AAS/AAO"]]

,,AAS/AAO
Biol.Rep,GroupLevel1,
1.0,G2.5Q2,0.297297
2.0,G2.5Q2,0.209266
3.0,G2.5Q2,0.228512
4.0,G2.5Q2,0.238417
1.0,G5Q2,0.236749
2.0,G5Q2,0.201644
3.0,G5Q2,0.196937
4.0,G5Q2,0.207459
1.0,G7Q2,0.300026


In [24]:
#Predictions

#[4-13C]GLC
DF["Pred_4gCit1"]=(DF["4gMalM1"]*DF["fMDHr_average"]*DF["fCS_average"]+DF["4gGlnM1"]*DF["fAAO_average"]*DF["fIDHr_average"])/(1-DF["fIDH_average"]*DF["fIDHr_average"])
DF["Pred_4gCit6"]=DF["4gMalM1"]*(1-DF["fMDHr_average"])*DF["fCS_average"] + DF["4gBicarb"]*DF["fIDHr_average"]
DF["Pred_4gCitP1D0"]=DF["Pred_4gCit1"]*0.5/DF["FCitDil_average"]
DF["Pred_4gCitP1D1"]=(DF["Pred_4gCit1"]*0.5+DF["Pred_4gCit6"])/DF["FCitDil_average"]

DF["Pred_4gGluM1"]=DF["Pred_4gCit1"]*DF["fIDH_average"]+DF["4gGlnM1"]*DF["fAAO_average"]


#[1,2-13C]GLN
DF["qqBicarbM0"] = 1-DF["qqBicarb"]
DF["qqGluM0"]=1-DF["qqGluM1"]-DF["qqGluM2"]-DF["qqGluM3"]

#[M+1]Citrate
DF["Pred_qqCit1"] = (DF["qqGluM2"] * DF["fTCA_average"] * 0.5 * DF["fCS_average"] + DF["qqBicarb"] * DF["fPC&MEr_average"] * (1-DF["fMDHr_average"]) * DF["fCS_average"]) / (1 - DF["fACLy_average"] * DF["fCS_average"] - DF["fIDH_average"]*DF["fIDHr_average"]*DF["qqBicarbM0"])
DF["Pred_qqCit6"] = (DF["qqGluM2"] * DF["fTCA_average"] * 0.5 * DF["fCS_average"] + DF["qqBicarb"] *DF["qqGluM0"]* DF["fIDHr_average"] + DF["qqBicarb"] * DF["fPC&MEr_average"] * DF["fMDHr_average"] * DF["fCS_average"]) / (1 - DF["fACLy_average"] * DF["fCS_average"])
DF["Pred_qqCitP1D0"] = 0.5 * DF["Pred_qqCit1"] / DF["FCitDil_average"]
DF["Pred_qqCitP1D1"] = (0.5 * DF["Pred_qqCit1"] + DF["Pred_qqCit6"]) / DF["FCitDil_average"]

#[M+2]Citrate
DF["Pred_qqCit12"] = (DF["qqGluM2"]*DF["qqBicarbM0"]*DF["fIDHr_average"]+DF["qqGluM3"]*DF["fTCA_average"]*DF["fCS_average"]*1/6)/(1-DF["fACLy_average"]*(1-DF["fMDHr_average"])*DF["fCS_average"])
DF["Pred_qqCit36"] = (DF["qqGluM3"]*DF["fTCA_average"]*DF["fCS_average"]*1/6 + DF["Pred_qqCit12"]*DF["fACLy_average"]*DF["fMDHr_average"]*DF["fCS_average"])/(1-DF["fACLy_average"]*DF["fCS_average"]- DF["fIDH_average"]*DF["fIDHr_average"]*DF["qqBicarb"])
DF["Pred_qqCit13"] = (DF["qqGluM3"] * DF["fTCA_average"] * 1/6 * DF["fCS_average"]) / (1 - DF["fACLy_average"] * DF["fCS_average"] - DF["fIDH_average"] * DF["fIDHr_average"]*DF["qqBicarbM0"])
DF["Pred_qqCit26"] = (DF["qqGluM3"]*DF["fTCA_average"]*DF["fCS_average"]*1/6) / (1 - DF["fACLy_average"] * DF["fCS_average"] - DF["fIDH_average"] * DF["fIDHr_average"] * DF["qqBicarb"])
DF["Pred_qqCit16"] = (DF["qqGluM3"] * DF["fTCA_average"] * 1/3 * DF["fCS_average"] + DF["Pred_qqCit1"] * DF["fIDH_average"] * DF["qqBicarb"] * DF["fIDHr_average"]) / (1 - DF["fIDH_average"] * DF["qqBicarb"] * DF["fIDHr_average"])

DF["Pred_qqMal34"] = DF["Pred_qqCit12"] * DF["fACLy_average"]*(1-DF["fMDHr_average"]) + DF["qqGluM3"]*DF["fTCA_average"]*1/6
DF["Pred_qqMal12"] = DF["Pred_qqCit12"] * DF["fACLy_average"]*DF["fMDHr_average"] + DF["qqGluM3"]*DF["fTCA_average"]*1/6 + DF["Pred_qqCit36"]*DF["fACLy_average"] 
DF["Pred_qqMal24"] = DF["qqGluM3"]*DF["fTCA_average"]*1/6 + DF["Pred_qqCit13"] * DF["fACLy_average"]
DF["Pred_qqMal13"] = DF["qqGluM3"]*DF["fTCA_average"]*1/6 + DF["Pred_qqCit26"] * DF["fACLy_average"]
DF["Pred_qqMal14"] = DF["qqGluM3"]*DF["fTCA_average"]*1/3 + DF["Pred_qqCit16"] * DF["fACLy_average"]

DF["Pred_qqCitP2D1"] = 0.5 * (DF["Pred_qqCit12"]+DF["Pred_qqCit16"]+DF["Pred_qqCit13"]) / DF["FCitDil_average"]
DF["Pred_qqCitP2D2"] = (0.5 * (DF["Pred_qqCit12"]+DF["Pred_qqCit16"]+DF["Pred_qqCit13"]) + (DF["Pred_qqCit36"]+DF["Pred_qqCit26"])) / DF["FCitDil_average"]


#[M+2]Precursor
DF["Pred_qqMalM2"] = (DF["Pred_qqMal34"]+DF["Pred_qqMal24"]+DF["Pred_qqMal14"]+DF["Pred_qqMal12"]+DF["Pred_qqMal13"])
DF["Pred_qqMalC1C4_ratio"]= (DF["Pred_qqMal34"]+DF["Pred_qqMal24"]+DF["Pred_qqMal14"])/DF["Pred_qqMalM2"]
DF["qqMalC1C4_ratio"]= DF["qqAspP2D1"]/DF["qqAspM2"]



#[5-13C]GLC
DF["5gGluM0"]=1-DF["5gGluM1"]-DF["5gGluM2"]
DF["5gMalM0"]=1-DF["5gMalM1"]-DF["5gMalM2"]
DF["5gACoAM0"]=1-DF["5g3PGM1"]
DF["Pred_5gMal4"]=DF["5gMalM1"]*DF["5gAspP1D0"]/DF["5gAspM1"]

DF["Pred_5gCit5"]=DF["5gMalM0"]*DF["5g3PGM1"]*DF["fCS_average"]+DF["5gGluP1D0"]*DF["fIDHr_average"]
DF["Pred_5gCit1"]=DF["Pred_5gMal4"]*DF["5gACoAM0"]*DF["fCS_average"]+DF["5gGluP1D1"]*DF["fIDHr_average"]
DF["Pred_5gCitP1D0"]=(DF["Pred_5gCit5"]+DF["Pred_5gCit1"])*0.5/DF["FCitDil_average"]



#Deviation from measured
DF["Dev#1"]=DF["Pred_4gCitP1D0"]/DF["4gCitP1D0"]
DF["Dev#2"]=DF["Pred_4gCitP1D1"]/DF["4gCitP1D1"]
DF["Dev#3"]=DF["Pred_4gGluM1"]/DF["4gGluM1"]
DF["Dev#5"]=DF["Pred_qqCitP1D0"]/DF["qqCitP1D0"]
DF["Dev#6"]=DF["Pred_qqCitP1D1"]/DF["qqCitP1D1"]
DF["Dev#7"]=DF["Pred_qqCitP2D1"]/DF["qqCitP2D1"]
DF["Dev#8"]=DF["Pred_qqCitP2D2"]/DF["qqCitP2D2"]
DF["Dev#9"]=DF["Pred_qqMalC1C4_ratio"]/DF["qqMalC1C4_ratio"]
DF["Dev#10"]=DF["Pred_5gCitP1D0"]/DF["5gCitP1D0"]




output2=DF.loc[:,["Dev#1","Dev#2","Dev#3","Dev#5","Dev#6","Dev#7","Dev#8","Dev#9","Dev#10"]]


In [25]:
DF.to_csv('Output_SteadyState_AllValues.csv')
output1 = DF.loc[:,["fIDHr","qqBicarb","FCitDil","fIDH","fAAO","fAAO_#2","fAADil","fPDH","fACLy","fPC&MEr","fTCA","fMDHr","4gBicarb","AAS/AAO"]]
output1.to_csv('Output_SteadyState_FractionalFluxes.csv')
output2.to_csv('Output_SteadyState_QualityControl.csv')

